In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
df = gpd.read_file('Leeds_LitterBins20211201.geojson')

In [ ]:
df['Ig_Date'] = pd.to_datetime(df['Ig_Date'], format='%Y')

In [3]:
df

,OBJECTID,ROAD_NAME,LOCATION,LOCATIONGROUP,EASTING,NORTHING,WARD,MODEL,APERTURE,FIXINGS,...,COMMENTS,LASTUPDATED,UPDATEDBY,FREQUENCY,SEQUENCE,CREATED_USER,CREATED_DATE,LAST_EDITED_USER,LAST_EDITED_DATE,geometry
0,3930,WHITEHALL ROAD,BUS STOP,BUS STOP OR SHELTER,429128.140000000013970,433098.540000000037253,HUNSLET & RIVERSIDE,METAL SQUARE,POSTBOX 4 SIDES WITH SELF CLOSING ASHTRAY,500MM X 500MM BALLAST ONLY,...,None,22/09/2020 00:00:00,None,DAILY,0.000000000000000,None,None,WAST,01/12/2021 15:32:00,POINT (-1.55931 53.79339)
1,3934,WHINMOOR,None,GENERAL PATH - HIGH FOOT FALL,436414.799999999988358,436522.010000000009313,CROSSGATES & WHINMOOR,METAL SQUARE,POSTBOX 4 SIDES WITH SELF CLOSING ASHTRAY,500MM X 500MM BALLAST ONLY,...,None,16/04/2019 00:00:00,None,MON/FRI,0.000000000000000,None,None,WAST,01/12/2021 15:32:00,POINT (-1.44830 53.82370)
2,3935,WHINMOOR CRESCENT,None,GENERAL PATH - HIGH FOOT FALL,435328.830000000016298,437991.669999999983702,CROSSGATES & WHINMOOR,METAL SQUARE,POSTBOX 4 SIDES WITH SELF CLOSING ASHTRAY,500MM X 500MM BALLAST ONLY,...,None,01/06/2019 00:00:00,None,DAILY,0.000000000000000,None,None,WAST,01/12/2021 15:32:00,POINT (-1.46463 53.83698)
3,3936,WHINMOOR WAY,OPP CO-OP,SHOPS/OFFICES,436756.710000000020955,436294.710000000020955,CROSSGATES & WHINMOOR,METAL SQUARE,POSTBOX 4 SIDES WITH SELF CLOSING ASHTRAY,500MM X 500MM BALLAST ONLY,...,None,01/06/2019 00:00:00,None,MON/FRI,0.000000000000000,None,None,WAST,01/12/2021 15:32:00,POINT (-1.44313 53.82163)
4,3937,WHINMOOR WAY,OPP PARK ENTRANCE,MONUMENT/BENCH/POW/CAR PARK,436499.229999999981374,436505.880000000004657,CROSSGATES & WHINMOOR,METAL SQUARE,POSTBOX 4 SIDES WITH SELF CLOSING ASHTRAY,500MM X 500MM BALLAST ONLY,...,None,01/06/2019 00:00:00,None,MON/FRI,0.000000000000000,None,None,WAST,01/12/2021 15:32:00,POINT (-1.44702 53.82355)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4163,3719,VICAR LANE,TASTE,SHOPS/OFFICES,430412.690000000002328,433803.790000000037253,LITTLE LONDON & WOODHOUSE,METAL SQUARE,POSTBOX 4 SIDES WITH SELF CLOSING ASHTRAY,500MM X 500MM BALLAST ONLY,...,None,22/09/2020 00:00:00,None,DAILY,0.000000000000000,None,None,WAST,01/12/2021 15:32:07,POINT (-1.53974 53.79965)
4164,3720,VICAR LANE,THE HIP STORE,SHOPS/OFFICES,430369.960000000020955,433565.950000000011642,LITTLE LONDON & WOODHOUSE,METAL SQUARE,POSTBOX 4 SIDES WITH SELF CLOSING ASHTRAY,500MM X 500MM BALLAST ONLY,...,None,22/09/2020 00:00:00,None,DAILY,0.000000000000000,None,None,WAST,01/12/2021 15:32:07,POINT (-1.54042 53.79752)
4165,3721,VICAR LANE,TRAILFINDERS,SHOPS/OFFICES,430396.450000000011642,433674.070000000006985,LITTLE LONDON & WOODHOUSE,METAL SQUARE,POSTBOX 4 SIDES WITH SELF CLOSING ASHTRAY,500MM X 500MM BALLAST ONLY,...,None,22/09/2020 00:00:00,None,DAILY,0.000000000000000,None,None,WAST,01/12/2021 15:32:07,POINT (-1.54000 53.79849)
4166,3722,VICAR LANE,CNR MERRION ST,GENERAL PATH - HIGH FOOT FALL,430416.080000000016298,433847.880000000004657,LITTLE LONDON & WOODHOUSE,METAL SQUARE,POSTBOX 4 SIDES WITH SELF CLOSING ASHTRAY,500MM X 500MM BALLAST ONLY,...,None,22/09/2020 00:00:00,None,DAILY,0.000000000000000,None,None,WAST,01/12/2021 15:32:07,POINT (-1.53969 53.80005)


In [ ]:
df['Ig_Date']

In [ ]:
def timestamp_to_year_part(df):
    date = df["ig_date"][0]
    year = int(date.to_pydatetime().strftime("%Y"))
    print('{} / {}'.format(date,year))
    df.year.values[:] = year
    return df

In [ ]:
df = df.assign(year=I32.type(0))
dfmeta = df._meta
df = df.map_partitions(timestamp_to_year_part, meta=dfmeta)